In [8]:
# Importing Modules
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial import distance
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.neighbors import NearestNeighbors

In [2]:
# Importing preprocessed and clean csv files
heart = pd.read_csv('heart_data_clean.csv')
sleep = pd.read_csv('sleep_data_clean.csv')

Below, we check the datasets to ensure that we have the correct ones to use for data-linkage

In [5]:
heart.head()

,id,age,gender,ap_hi,ap_lo,ap,active,bmi_level,cardio,height,weight,bmi,cholesterol,gluc,smoke,alco
0,0,50,2,110,80,30,1,1,0,168,62.0,21.967120,1,1,0,0
1,1,55,1,140,90,50,1,3,1,156,85.0,34.927679,3,1,0,0
2,2,52,1,130,70,60,0,1,1,165,64.0,23.507805,3,1,0,0
3,3,48,2,150,100,50,1,2,1,169,82.0,28.710479,1,1,0,0
4,4,48,1,100,60,40,0,1,0,156,56.0,23.011177,1,1,0,0


In [6]:
heart.describe()

,id,age,gender,ap_hi,ap_lo,ap,active,bmi_level,cardio,height,weight,bmi,cholesterol,gluc,smoke,alco
count,68034.000000,68034.000000,68034.000000,68034.000000,68034.000000,68034.000000,68034.000000,68034.000000,68034.000000,68034.000000,68034.000000,68034.000000,68034.000000,68034.000000,68034.000000,68034.000000
mean,49975.611003,53.285945,1.350046,126.489946,81.208205,45.281741,0.803422,1.890334,0.492930,164.471941,73.718110,27.276698,1.362142,1.223521,0.088132,0.053297
std,28846.843008,6.764880,0.476987,16.401575,9.303021,11.537754,0.397414,0.783774,0.499954,7.838807,13.552824,4.856586,0.676979,0.569252,0.283489,0.224627
min,0.000000,30.000000,1.000000,60.000000,30.000000,0.000000,0.000000,1.000000,0.000000,120.000000,11.000000,3.471784,1.000000,1.000000,0.000000,0.000000
25%,25000.500000,48.000000,1.000000,120.000000,80.000000,40.000000,1.000000,1.000000,0.000000,159.000000,65.000000,23.875115,1.000000,1.000000,0.000000,0.000000
50%,50016.500000,54.000000,1.000000,120.000000,80.000000,40.000000,1.000000,2.000000,0.000000,165.000000,72.000000,26.291724,1.000000,1.000000,0.000000,0.000000
75%,74869.750000,58.000000,2.000000,140.000000,90.000000,50.000000,1.000000,3.000000,1.000000,170.000000,82.000000,30.083829,1.000000,1.000000,0.000000,0.000000
max,99999.000000,65.000000,2.000000,200.000000,125.000000,135.000000,1.000000,3.000000,1.000000,250.000000,149.000000,44.997166,3.000000,3.000000,1.000000,1.000000


In [9]:
heart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68034 entries, 0 to 68033
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           68034 non-null  int64  
 1   age          68034 non-null  int64  
 2   gender       68034 non-null  int64  
 3   ap_hi        68034 non-null  int64  
 4   ap_lo        68034 non-null  int64  
 5   ap           68034 non-null  int64  
 6   active       68034 non-null  int64  
 7   bmi_level    68034 non-null  int64  
 8   cardio       68034 non-null  int64  
 9   height       68034 non-null  int64  
 10  weight       68034 non-null  float64
 11  bmi          68034 non-null  float64
 12  cholesterol  68034 non-null  int64  
 13  gluc         68034 non-null  int64  
 14  smoke        68034 non-null  int64  
 15  alco         68034 non-null  int64  
dtypes: float64(2), int64(14)
memory usage: 8.3 MB


In [7]:
sleep.head()

,id,age,gender,ap_hi,ap_lo,ap,active,bmi_level,sleep_duration,quality_sleep,p_active_level,stress_level,heart_rate,steps,occupation
0,1,41,2,126,82,44,0,2,10,8.0,1,8,95,7000,Software Engineer
1,2,44,2,138,74,64,1,2,8,7.0,1,6,66,9000,Accountant
2,3,31,2,123,68,55,0,3,8,7.0,3,5,76,4000,Salesperson
3,4,36,2,131,76,55,0,2,10,8.0,2,6,74,6000,Sales Representative
4,5,37,2,103,67,36,0,2,10,8.0,3,3,89,7000,Lawyer


In [8]:
sleep.describe()

,id,age,gender,ap_hi,ap_lo,ap,active,bmi_level,sleep_duration,quality_sleep,p_active_level,stress_level,heart_rate,steps
count,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,500.500000,35.134000,1.50000,120.273000,79.849000,40.424000,0.553000,2.019000,6.932000,6.466000,2.953000,4.944000,75.140000,7933.000000
std,288.819436,9.676036,0.50025,9.974767,7.871478,12.925219,0.497432,0.831463,1.506534,0.753267,1.002894,2.006711,10.096353,1984.048247
min,1.000000,5.000000,1.00000,90.000000,58.000000,3.000000,0.000000,1.000000,2.000000,4.000000,0.000000,0.000000,44.000000,2000.000000
25%,250.750000,28.000000,1.00000,114.000000,75.000000,32.000000,0.000000,1.000000,6.000000,6.000000,2.000000,4.000000,68.000000,7000.000000
50%,500.500000,35.000000,1.50000,120.000000,80.000000,40.500000,1.000000,2.000000,7.000000,6.500000,3.000000,5.000000,75.000000,8000.000000
75%,750.250000,41.000000,2.00000,127.000000,85.000000,49.250000,1.000000,3.000000,8.000000,7.000000,4.000000,6.000000,82.000000,9000.000000
max,1000.000000,67.000000,2.00000,150.000000,103.000000,77.000000,1.000000,3.000000,11.000000,8.500000,6.000000,10.000000,105.000000,15000.000000


In [10]:
sleep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              1000 non-null   int64  
 1   age             1000 non-null   int64  
 2   gender          1000 non-null   int64  
 3   ap_hi           1000 non-null   int64  
 4   ap_lo           1000 non-null   int64  
 5   ap              1000 non-null   int64  
 6   active          1000 non-null   int64  
 7   bmi_level       1000 non-null   int64  
 8   sleep_duration  1000 non-null   int64  
 9   quality_sleep   1000 non-null   float64
 10  p_active_level  1000 non-null   int64  
 11  stress_level    1000 non-null   int64  
 12  heart_rate      1000 non-null   int64  
 13  steps           1000 non-null   int64  
 14  occupation      1000 non-null   object 
dtypes: float64(1), int64(13), object(1)
memory usage: 117.3+ KB


Now that we have checked our datasets, we will start with the data-linkage process. First, we will have to compute the distances/similarity measures to see how similar each unit in the sleeping data is to units in the heart dataset. After that, we will proceed to Fuzzy Matching (a.k.a. fuzzy logic) to help us identify units that are similar but not equally the same by using the same feature columns that exist in both datasets (i.e. age, gender, blood pressure rates, and activity levels). We will evaluate the difference in the metrics present and look into how similar each unit in the heart dataset is compared to each particular unit in the sleep dataset (using sleep dataset as the reference because it is smaller in size, meaning that we will match each unit from the sleep dataset with a unit from the heart dataset and discard the remaining larger heart dataset units).

In [32]:
""" Determining relevant parameters from both datasets for us to compare and determine 
similarity between units from both datasets."""

relevantParams = ['age', 'gender', 'ap_hi', 'ap_lo', 'ap', 'active', 'bmi_level']

# Filtering out unnecessary columns only for comparison stage
heartRelevant = heart[relevantParams]
sleepRelevant = sleep[relevantParams]

# Normalizing the data before performing Euclidean Distance
scaler = MinMaxScaler()
heartScaled = scaler.fit_transform(heartRelevant)
sleepScaled = scaler.transform(sleepRelevant)

heartScaled = pd.DataFrame(heartScaled, columns=relevantParams)
sleepScaled = pd.DataFrame(sleepScaled, columns=relevantParams)

In [33]:
# Sampling n samples from heart dataset where n = size of sleep dataset (1,000)
heartSampled = heartScaled.sample(n=len(sleepScaled), random_state=1)

# Performing sample distances for global sigma below (used for Fuzzy matching later)
sampleDistances = euclidean_distances(sleepScaled, heartSampled)

# Estimate global sigma from the sampled distances
sigmaGlobal = np.std(sampleDistances)

In [34]:
# Method 1: Using Gaussian Membership functions to perform fuzzy logic

gaussianLinkage = []

# We want to find the 5 nearest neighbors, so k = 5
k = 5

# Fitting nn on sampled heart dataset
nbrs = NearestNeighbors(n_neighbors=k, algorithm='ball_tree').fit(heartSampled)

# Finding Knn in sampled heart dataset for each unit in sleep dataset
distances, indices = nbrs.kneighbors(sleepScaled)

# Looping over each record in sleep to find 5 nearest heart units
for indA, (dist, idx) in enumerate(zip(distances, indices)):
    # Applying Gaussian Membership Func
    scores = np.exp(-((dist**2) / (2 * sigmaGlobal**2)))
    bestScoreInd = np.argmax(scores)
    bestSampleMatchInd = idx[bestScoreInd]
    linkageScore = scores[bestScoreInd]

    # Mapping back to original index in heart dataset
    originalHeartIndex = heartSampled.iloc[bestSampleMatchInd].name
    
    # Save the linkage
    gaussianLinkage.append({'SleepIndex': indA, 'HeartIndex': originalHeartIndex, 'Score': linkageScore})

# Convert linkages to a DataFrame
gaussianLinkageDf = pd.DataFrame(gaussianLinkage)

In [35]:
print(f"The Number of Units that had a similarity score of above 0.80: {(gaussianLinkageDf['Score'] > 0.80).sum()}")

The Number of Units that had a similarity score of above 0.80: 600


The method above utilizes Gaussian in order to find similarity between sleep units to a randomly selected batch of 1000 units from the heart dataset. One thing to consider is that it is comparing it to a subset of samples from heart, which may limit it to finding closer matches outside the subset bubble. However, we have set a fuzzy score threshold of around 0.80 similarity and have gotten around 60 data points out of 1000 that pass this threshold. There exists an accuracy/complexity tradeoff and we want to see if running on smaller subset of 1000 is worth it for computational power.

We can try another method like batching to see if we can compare to entire set of heart dataset to see if we can bring that number up from 607 to a potentially higher number

In [41]:
# Choosing batch size of 10000
batchSize = 10000

# Calculate the number of batches needed
batchNums = np.ceil(len(heartScaled) / batchSize)

sigmaBatch = 1
batchLinkage = []

# Looping through the batches of size 10000 each
for batchNum in range(int(batchNums)):
    start = batchNum * batchSize
    end = min((batchNum + 1) * batchSize, len(heartScaled))
    
    # Choosing current batch of 10000 from heart dataset
    heartBatch = heartScaled[start:end]
    
    # Finding Euclidean distance between sleep and current heart data batch
    distances = euclidean_distances(sleepScaled, heartBatch)

    # Convert distances to fuzzy similarity scores using a Gaussian membership function
    scores = np.exp(-((distances ** 2) / (2 * sigmaBatch ** 2)))
    
    # Finding minimum distance (which represents highest similarity)
    for i, score in enumerate(scores):
        bestMatchInd = np.argmax(score)
        bestMatch = score[bestMatchInd]
        originalHeartIndex = start + bestMatchInd
        batchLinkage.append({'sleepIndex': i, 'heartIndex': originalHeartIndex, 'Score': bestMatch})

batchLinkageDf = pd.DataFrame(batchLinkage)

Because we have performed the above fuzzy logic with batches, our batch linkage dataframe will feature more than 1000 (the size of sleep dataframe) fuzzy scores, because we will have more than one heart unit attached to the same sleep unit. We will need to remove the redundant values that scored less so that only one best heart unit result is linked to each sleep result.

In [47]:
# Grouping by the Sleep Index and only keeping the highest fuzzy score for each 
filteredResult = batchLinkageDf.loc[batchLinkageDf.groupby('sleepIndex')['Score'].idxmax()]
filteredResult.reset_index(drop=True, inplace=True)
# Checking that it went down to 1000 total units
filteredResult.shape

(1000, 3)

In [55]:
print(f"The Number of Units that had a similarity score of above 0.80: {(filteredResult['Score'] > 0.80).sum()}")

The Number of Units that had a similarity score of above 0.80: 986


Since we have a high score of 986 units above 0.80 (meaning that batch is much better than sampled), let us check for 0.90 while we are at it.

In [56]:
print(f"The Number of Units that had a similarity score of above 0.90: {(filteredResult['Score'] > 0.90).sum()}")

The Number of Units that had a similarity score of above 0.90: 904


Our result is 904. Using a 90% Fuzzy Match threshold is much safer for data linkages, even if it comes at the cost of having around 82 less units from 986. 904 units out of 1000 that have above a 90% Fuzzy score is very good.

In [60]:
# Removing fuzzy scores below 90%.
resultsOver90 = filteredResult[filteredResult['Score'] > 0.90]
resultsOver90.shape

(904, 3)

In [61]:
resultsOver90.head()

,sleepIndex,heartIndex,Score
0,0,30354,0.999318
1,1,750,0.998902
2,2,43445,0.973109
3,3,45060,0.991905
4,4,3987,0.994156


In [119]:
sleepIndList = resultsOver90['sleepIndex'].tolist()
heartIndList = resultsOver90['heartIndex'].tolist()

nRows = resultsOver90.shape[0]

# Creating empty combined dataframe to fill
combinedDf = pd.DataFrame({'age': [np.nan] * nRows,
                   'gender': [np.nan] * nRows,
                   'ap_hi': [np.nan] * nRows,
                   'ap_lo': [np.nan] * nRows,
                   'ap': [np.nan] * nRows,
                   'active': [np.nan] * nRows,
                   'bmi_level': [np.nan] * nRows,
                   'cardio': [np.nan] * nRows,
                   'height': [np.nan] * nRows,
                   'weight': [np.nan] * nRows,
                   'bmi': [np.nan] * nRows,
                   'cholesterol': [np.nan] * nRows,
                   'gluc': [np.nan] * nRows,
                   'smoke': [np.nan] * nRows,
                   'alco': [np.nan] * nRows,
                   'sleep_duration': [np.nan] * nRows,
                   'quality_sleep': [np.nan] * nRows,
                   'p_active_level': [np.nan] * nRows,
                   'stress_level': [np.nan] * nRows,
                   'heart_rate': [np.nan] * nRows,
                   'steps': [np.nan] * nRows,
                   'occupation': [np.nan] * nRows})

combinedDf.head()

,age,gender,ap_hi,ap_lo,ap,active,bmi_level,cardio,height,weight,...,gluc,smoke,alco,sleep_duration,quality_sleep,p_active_level,stress_level,heart_rate,steps,occupation
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [129]:
# Converitng respective indices to list format
sleepIndList = resultsOver90['sleepIndex'].tolist()
heartIndList = resultsOver90['heartIndex'].tolist()

# Filling in the empty rows
for idx, row, in combinedDf.iterrows():
    combinedDf.at[idx, 'age'] = pd.Series([sleep.loc[sleepIndList[idx], 'age'], heart.loc[heartIndList[idx], 'age']]).mean().round(0).astype(int)
    combinedDf.at[idx, 'gender'] = pd.Series([sleep.loc[sleepIndList[idx], 'gender'], heart.loc[heartIndList[idx], 'gender']]).mean().round(0).astype(int)
    combinedDf.at[idx, 'ap_hi'] = pd.Series([sleep.loc[sleepIndList[idx], 'ap_hi'], heart.loc[heartIndList[idx], 'ap_hi']]).mean().round(0).astype(int)
    combinedDf.at[idx, 'ap_lo'] = pd.Series([sleep.loc[sleepIndList[idx], 'ap_lo'], heart.loc[heartIndList[idx], 'ap_lo']]).mean().round(0).astype(int)
    combinedDf.at[idx, 'ap'] = pd.Series([sleep.loc[sleepIndList[idx], 'ap'], heart.loc[heartIndList[idx], 'ap']]).mean().round(0).astype(int)
    combinedDf.at[idx, 'active'] = pd.Series([sleep.loc[sleepIndList[idx], 'active'], heart.loc[heartIndList[idx], 'active']]).mean().round(0).astype(int)
    combinedDf.at[idx, 'bmi_level'] = pd.Series([sleep.loc[sleepIndList[idx], 'bmi_level'], heart.loc[heartIndList[idx], 'bmi_level']]).mean().round(0).astype(int)
    combinedDf.at[idx, 'cardio'] = heart.loc[heartIndList[idx], 'cardio'].astype(int)
    combinedDf.at[idx, 'height'] = heart.loc[heartIndList[idx], 'height'].astype(int)
    combinedDf.at[idx, 'weight'] = heart.loc[heartIndList[idx], 'weight'].astype(int)
    combinedDf.at[idx, 'bmi'] = heart.loc[heartIndList[idx], 'bmi']
    combinedDf.at[idx, 'cholesterol'] = heart.loc[heartIndList[idx], 'cholesterol'].astype(int)
    combinedDf.at[idx, 'gluc'] = heart.loc[heartIndList[idx], 'gluc'].astype(int)
    combinedDf.at[idx, 'smoke'] = heart.loc[heartIndList[idx], 'smoke'].astype(int)
    combinedDf.at[idx, 'alco'] = heart.loc[heartIndList[idx], 'alco'].astype(int)
    combinedDf.at[idx, 'sleep_duration'] = sleep.loc[sleepIndList[idx], 'sleep_duration'].astype(int)
    combinedDf.at[idx, 'quality_sleep'] = sleep.loc[sleepIndList[idx], 'quality_sleep']
    combinedDf.at[idx, 'p_active_level'] = sleep.loc[sleepIndList[idx], 'p_active_level'].astype(int)
    combinedDf.at[idx, 'stress_level'] = sleep.loc[sleepIndList[idx], 'stress_level'].astype(int)
    combinedDf.at[idx, 'heart_rate'] = sleep.loc[sleepIndList[idx], 'heart_rate'].astype(int)
    combinedDf.at[idx, 'steps'] = sleep.loc[sleepIndList[idx], 'steps'].astype(int)
    combinedDf.at[idx, 'occupation'] = sleep.loc[sleepIndList[idx], 'occupation']

In [131]:
combinedDf.head()

,age,gender,ap_hi,ap_lo,ap,active,bmi_level,cardio,height,weight,...,gluc,smoke,alco,sleep_duration,quality_sleep,p_active_level,stress_level,heart_rate,steps,occupation
0,40.0,2.0,126.0,81.0,44.0,0.0,2.0,1.0,180.0,94.0,...,1.0,0.0,1.0,10.0,8.0,1.0,8.0,95.0,7000.0,Software Engineer
1,44.0,2.0,139.0,76.0,63.0,1.0,2.0,1.0,179.0,87.0,...,1.0,1.0,1.0,8.0,7.0,1.0,6.0,66.0,9000.0,Accountant
2,35.0,2.0,122.0,69.0,52.0,0.0,3.0,1.0,176.0,105.0,...,1.0,0.0,0.0,8.0,7.0,3.0,5.0,76.0,4000.0,Salesperson
3,38.0,2.0,130.0,78.0,52.0,0.0,2.0,0.0,170.0,82.0,...,1.0,1.0,0.0,10.0,8.0,2.0,6.0,74.0,6000.0,Sales Representative
4,38.0,2.0,106.0,68.0,38.0,0.0,2.0,0.0,166.0,70.0,...,1.0,0.0,0.0,10.0,8.0,3.0,3.0,89.0,7000.0,Lawyer


In [127]:
# sleepIndList = resultsOver90['sleepIndex'].tolist()
# heartIndList = resultsOver90['heartIndex'].tolist()

# # Changing column names to make sure there exists fully unique columns before merging
# renamedColsSleep = sleep.add_suffix('_sleep')
# renamedColsHeart = heart.add_suffix('_heart')

# selectedRowsSleep = renamedColsSleep.loc[sleepIndList]
# selectedRowsHeart = renamedColsHeart.loc[heartIndList]

# allCols = renamedColsSleep.columns.tolist() + renamedColsHeart.columns.tolist()
# combinedDf = pd.DataFrame(columns=allCols)

# # Looping over sleep and heart using their respective indices selected to combine together
# for i, j in zip(selectedRowsSleep, selectedRowsHeart):
#     sleepRow = renamedColsSleep.iloc[[i]].reset_index(drop=True)
#     heartRow = renamedColsHeart.iloc[[j]].reset_index(drop=True)

#     mergedRow = pd.concat([sleepRow, heartRow], axis=1)
#     combinedDf = pd.concat([combinedDf, mergedRow], ignore_index=True)

# combinedDf.head()

In [128]:
# # Merge duplicate columns together
# combinedDf['age'] = combinedDf[['age_sleep', 'age_heart']].mean(axis=1).round(0)
# combinedDf['gender'] = combinedDf[['gender_sleep', 'gender_heart']].mean(axis=1).round(0)
# combinedDf['ap_hi'] = combinedDf[['ap_hi_sleep', 'ap_hi_heart']].mean(axis=1).round(0)
# combinedDf['ap_lo'] = combinedDf[['ap_lo_sleep', 'ap_lo_heart']].mean(axis=1).round(0)
# combinedDf['ap'] = combinedDf[['ap_sleep', 'ap_heart']].mean(axis=1).round(0)
# combinedDf['active'] = combinedDf[['active_sleep', 'active_heart']].mean(axis=1).round(0)
# combinedDf['bmi_level'] = combinedDf[['bmi_level_sleep', 'bmi_level_heart']].mean(axis=1).round(0)

# # Dropping Duplicate Columns
# combinedDf.drop(['age_sleep', 'age_heart', 'gender_sleep', 'gender_heart', 
#                 'ap_hi_sleep', 'ap_hi_heart', 'ap_lo_sleep', 'ap_lo_heart',
#                 'ap_sleep', 'ap_heart', 'active_sleep', 'active_heart',
#                 'bmi_level_sleep', 'bmi_level_heart'], axis=1, inplace=True)

# # Moving the duplicate columns first while keeping rest in same order
# orderingParams = ['age', 'gender', 'ap_hi', 'ap_lo', 'ap', 'active', 'bmi_level']
# changedOrder = orderingParams + [col for col in combinedDf.columns if col not in orderingParams]
# resultDf = combinedDf[changedOrder]

# resultDf.reset_index(inplace=True)
# resultDf.rename(columns={'index': 'id'}, inplace=True)

In [132]:
# Converting Results to csv file
combinedDf.to_csv('resultantDataset.csv', index=False)